# Collecting Data from an old Heating System
## First Steps
Due to the climate and energy crisis, something had to be done to reduce our fuel consumption. We have a 27 year old gas heating system with a simple control system. One of our first actions was to turn the heating off when we were away for several days and turn it back on in time for our return. We bought a switchable plug (Shelly Plug) and a WLAN-enabled temperature sensor (Shelly H&T). This enabled us to keep an eye on the temperature in the house via the corresponding APP and to activate the heating if necessary on cold winter days.
This gave us an easy way to save energy without sacrificing comfort.

<table>
  <tr>
    <th><a href="https://github.com/Sepp28/" target="_blank"><img src="./images/Shelly_Temp_3.jpg" 
            alt="Link to ..." width="240" height="180" border="10" /></a></th>
    <th><a href="https://github.com/Sepp28/" target="_blank"><img src="./images/Shelly_Power_01.jpg" 
            alt="Link to ..." width="240" height="180" border="10" /></a></th>
  </tr>
  <tr>
    <td><center>Indoor temperature during a long absence</center></td>
    <td><center>Electrical power consuption</center></td>
  </tr>
</table>

As  shown in the left diagram in the beginning the temperature is constantly decreasing until it reaches a lower limit of about 10°C. After the 24th it rises within about 5 days due to a heat up in preparationof our return. The diagramm at right shows to current consumption of the heating system for control and water pumps. It was activated twice in the middle of the month to prevent the temperature to fall below ~10°C and a constant operation during heatup. The hight of the power graph indicates the amout of heating.

## The Discovery
By checking the indoor temperature from a distance, we noticed that the heating did not always behave as expected. Therefore, we analyzed the electricity consumption of the heating in more detail over longer time periods and were surprised how well we could understand what the heating was doing. We discovered situations where the heating was running very inefficiently.
Therefore, we first had to systematically collect data so that, in a further step, we could preferentially activate the heating when it can run very efficiently.

## The Python Code for Data Logging with a Raspberry Pi
The data is collected via an HTTP request over the home WLAN. Since this is done 24 hours a day/7 days a week, an old Rasperry Pi was used, which permanently documents the power flow via the Shelly Plug in a csv file.    
With this notebook the code can be tested easily. A pure Python code is then exported for the Raspberry.

In [1]:
# import the necessary lybraries
import requests
import json
import time
import sys
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from webdav3.client import Client # only necessary, if the data is loaded in parallel to a web server.

In [2]:
options = {
    # insert the data for your cloud storage
    'webdav_hostname': "https://mediacenter.gmx.net/",
    'webdav_login':    "josef.krammer@gmx.net",
    'webdav_password': "gmx1gmx1",
    'cert_path':       "/etc/ssl/certs/ca-certificate.crt"
}
client = Client(options)

The code must run for a very long time and reliably. Therefore it is tested before it is installed on the Raspberry. The test also includes error cases. It runs with shorter time constants. So it can be tested within a short time, what would need days and weeks in reality. The test/normal mode is controlled by a test flag (*test_flag*).

In [3]:
# Now we set some variables. Dependent on test mode or normal mode.

test_flag = False                # Switch to change to select test or normal mode.
ip_local = '192.168.178.'
if test_flag==False: # no test/ normal operation
    ip_number        = 47        # last number of the ip-address (plug that connects the heating)
    samples_per_file = 1800      # periodically samples are written to a individual csv file
    print_factor     = 100       # a prompt appears in shorter intervalls to see the activities
    sleep_relay_off  = 10        # when the plug is switched off the http request is done in longer intervals
    sub_directory    = 'Shelly/' # subdirectory to save the data
    sub_directory_wrong = 'Shelly/' # an alternative subdirectory to save the data
    number_wrong_cloud_transfers = -1
    text             = 'normal operation'
else:                # during test the parameters are different
    ip_number        = 52        # number ip-address of an alternative plug to not disturb the normal operation
    samples_per_file = 10        # reduced number of samples per file
    print_factor     = 2         # print the prompt more often
    sleep_relay_off  = 1         # saple periode during off 
    sub_directory    = 'Test/'   # an alternative subdirectory to save test data
    sub_directory_wrong = 'Test_wrong/' # a non existing subdirectory to cause a fault
    number_wrong_cloud_transfers = 2    #
    text             = 'test'

ip_address = ip_local+str(ip_number)    # complete ip address
print(text+' with ip address:', ip_address)

normal operation with ip address: 192.168.178.47


### Now we define some usefull functions

In [4]:
# convert a time stamp (eg. 1679929979) to a time string (eg. 2023-Mar-27 15:12:59)
def get_time_string(time_stamp):
    return datetime.utcfromtimestamp(time_stamp).strftime('%Y-%b-%d %H:%M:%S')

In [5]:
# makes an http request and repeats it (max_tries) times in 5s intervals
def request_n_times(url, max_tries):
    i = 0
    while i < max_tries:
        i += 1
        r = None
        try:
            r = requests.get(url,timeout=5)
            if r.status_code != 200:
                raise Exception('request status_code != 200')
        except Exception as e:
            if r is None:
                status_code = 'None'
                #print('\n',datetime.now(),'status-code:', r.status_code)
            else:
                status_code = r.status_code
            print('\n', datetime.now(), 'HTTP status', status_code, 'retry:', i)
            print(e)
            time.sleep(5)
        else:
            return r
    print('maximum retrials exceeded', i)
    return r

In [6]:
# sends a HTTP request tu the Shelly Plug with an unlimited number of retries.
# and returns the parameters in a list.
def request_shelly_plug_data(ip_address):
    r = request_n_times('http://'+ip_address+'/status', 10) # repeats 10 request every 5 s
    while r == None:                                        # pauses after 10 trials for 30s
        print('Wait for 30s an try it again')
        time.sleep(30)
        r = request_n_times('http://'+ip_address+'/status', 10)
    r_j = json.loads(r.text)
    unix_t = r_j['unixtime']
    timestamp = r_j['meters'][0]['timestamp']
    power = r_j['meters'][0]['power']
    total = r_j['meters'][0]['total']
    relay = r_j['relays'][0]['ison']
    temperature = r_j['temperature']
    over_temperature = r_j['overtemperature']
    uptime = r_j['uptime']
    return [unix_t, timestamp, power, total, relay, temperature, over_temperature, uptime]

In [7]:
# retrieves data from the Shelly and prints some of it to the console
def print_shelly_plug_data(ip_address):
    data = request_shelly_plug_data(ip_address)
    relay = 'off'
    if data[4]==True:
        relay='on'
    uptime_str = timedelta(data[7]/(24*3600))
    print('{}: {}W (total: {:.3f}kWh), Relay={}, Temp={}, Uptime: {}'.
          format(get_time_string(data[1]),data[2],data[3]/60000, relay, data[5], uptime_str))
    #print(data)

In [8]:
# Write a bunch of data samples to a csv file in the subdirectory 'data'
# the file name is 'Shelly_II_YYMMDD_hhmmss.csv', eg.: 'Shelly_47_230327_151259.csv'
# Hence all file names are in ascending order.
def write_data_to_file(unit_name,rec_time,power,total,temp):
    data1 = {'timestamp': rec_time, 'power': power, 'total': total, 'temperature': temp}
    data_frame = pd.DataFrame(data1)
    file_name = 'data/Shelly_'+unit_name+'_'+datetime.utcfromtimestamp(data[1]).strftime('%y%m%d_%H%M%S')+'.csv'
    data_frame.to_csv(file_name, index=False)
    return file_name

In [9]:
# Now we try a http request and print the result
print_shelly_plug_data(ip_address)

2023-Mar-29 13:00:25: 14.36W (total: 28.488kWh), Relay=on, Temp=28.6, Uptime: 55 days, 0:15:55


## Upload the Files to the Cloud
In order to have a global access to the files, they are uploaded to a cloud storage. This is done regularely to have a short term insight. In case of a termination the data is almost completely secured. For that we have created a Uploader class that also encapsulates the complexity to make retries in case the uploading sporadically goes wrong.

In [10]:
class Uploader():
    def __init__(self, client, sub_directory, sub_directory_wrong, number_wrong_cloud_transfers):
        self.client = client
        #self.ip_str = ip_address.split('.')[3]
        self.sub_directory = sub_directory
        self.sub_directory_wrong = sub_directory_wrong
        self.number_wrong_cloud_transfers = number_wrong_cloud_transfers
        self.n_files   = 0  # reset counter for number of files written
        self.ind_wrong = 0  # counter of retries to transfer a file to the cloud
        self.not_uploaded_remote_file_name = None
        self.not_uploaded_local_file_name  = None
        self.number_uploaded_files = 0
        self.number_upload_retries = 0

    def upload(self,file_name):
        if self.ind_wrong<self.number_wrong_cloud_transfers:
            self.ind_wrong += 1
            remote_file_name = self.sub_directory_wrong+file_name.split('/')[len(file_name.split('/'))-1]
        else:
            remote_file_name = self.sub_directory+file_name.split('/')[len(file_name.split('/'))-1]
        try:
            self.client.upload_sync(remote_path=remote_file_name, local_path=file_name)
            self.number_uploaded_files += 1
        except Exception as e:
            print(datetime.now(), 'Exception:',type(e), e)# '\n', remote_file_name)
            #print(e)
            if self.not_uploaded_local_file_name!=None:
                print('######### file:', self.not_uploaded_local_file_name, 'not uploaded! ###########')
            self.not_uploaded_remote_file_name = remote_file_name
            self.not_uploaded_local_file_name  = file_name
            self.number_upload_retries = 0
        
        
    def retry_upload(self): # in case something prevented the last upload. rety it now.
        if self.not_uploaded_local_file_name != None: # retry to upload the not yet uploaded file
            try:
                self.client.upload_sync(remote_path=not_uploaded_remote_file_name, 
                                   local_path=not_uploaded_local_file_name)
                self.number_uploaded_files += 1
                self.not_uploaded_remote_file_name = None
                self.not_uploaded_local_file_name  = None            
            except Exception as e:
                print(datetime.now(), 'Exception rep.:',type(e), e)# '\n', self.not_uploaded_remote_file_name)
                self.number_upload_retries += 1
        if self.number_upload_retries >= 20:
            print('######### file:', self.not_uploaded_local_file_name, 'not uploaded! ###########')
            self.number_upload_retries = 0
            self.not_uploaded_remote_file_name = None
            self.not_uploaded_local_file_name  = None


## Save Data in Chunks to csv Files
As shown in the left image below 4 values are recorded for each sample.
- tmestamp: unix encoded time at which the values were measured
- power:    power in Watts at measurement time
- total: Energy in Wh since the last reset of the plug.
- temperature: internal temperature of the plug. (Ambient temperature plus self-heating)

Thr right image shows a sequence of files recorded on March 14th. Please note the file names that contain the save time.

<table>
  <tr>
    <th><a href="https://github.com/Sepp28/" target="_blank"><img src="./images/table_01.png" 
            alt="Link to ..." width="360" height="240" border="10" /></a></th>
    <th><a href="https://github.com/Sepp28/" target="_blank"><img src="./images/files_03.png" 
            alt="Link to ..." width="360" height="240" border="10" /></a></th>
  </tr>
  <tr>
    <td><center>Example of a csv file</center></td>
    <td><center>csv files in the subdirectory</center></td>
  </tr>
</table>



## The Main Loop
The loop runs indefinitely. Data is fetched from the connector. When the defined number of data has been collected, they are saved to a file. In slightly shorter intervals a message about the current state of the data and files is output via console. The loop runs with one second when the relay is closed and ten seconds (=*sleep_relay_off*) when the relay is open, because in this case the power values are zero.

In [ ]:
# Define Lists for the parameters that have to be recorded
rec_time  = [] # recording time = time stamp
power     = [] # Power of the sample
total     = [] # total accumulated power
temp      = [] # internal temparature of the Shelly Plug

ind       = 0  # counter for the number of samples needed for a file
uploader = Uploader(client, sub_directory, sub_directory_wrong, number_wrong_cloud_transfers)
n_files   = 0  # number of files saved to disk

while True:
    data = request_shelly_plug_data(ip_address)
    power.append(data[2])
    total.append(data[3])
    temp.append(data[5])
    rec_time.append(data[1])
    relay = data[4]
    ind += 1
    if ind==samples_per_file:
        file_name = write_data_to_file(ip_str,rec_time,power,total,temp)
        n_files += 1
        uploader.upload(file_name)
        power    = []
        total    = []
        temp     = []
        rec_time = []
        ind = 0
    if ind%print_factor==0: # print after n=print_factor samples
        print("{:4}/{}. files disk {} cloud {}".format(ind,samples_per_file, n_files,
                                                       uploader.number_uploaded_files))
        sys.stdout.flush()
        uploader.retry_upload()
    if relay:
        time.sleep(1)
    else:
        time.sleep(sleep_relay_off)


## Hardware
Except for the plug, an old Raspberry Pi Model B is used. It requires only a few watts of supply power. Since it is a full Linux machine, it can run Python programs. Remote operation is possible via SSH. Therefore, the RasPi can be positioned anywhere within the range of the WLAN and does not need a screen or keyboard. A USB power supply with 2A maximum current is used for stable continuous operation. Unlike with the previous 1A power supply, it has been running stably for several weeks now. 


## Outlook
The following diagram is a preview of a more detailed analysis as shown in the next notebook. The picture shows the electrical power curve of the gas heater.  The heater is turned on at about 7:00 and turned off at 21.00. The peaks to over 400W are caused by the glow igniter to start the gas burner. It can be seen that the gas burner is turned on and off cyclically during the day. The power consumption for the pumps, valves, etc. is well below 100W. Immediately after start and twice after 18:00 you can see a small pause in the cycle as well as a phase that requires about 50W of power. These are phases when the hot water boiler is heated up. The slowly lengthening cycle during the day is also noticeable. This results from the rising outdoor temperature and thus reduced heating power. A more detailed analysis will follow.


[![Test.png](./images/plot_02.png)](https://github.com/Sepp28/)
<center>Power consuption of the heating system during one day</center>
